In [ ]:
# Hierarchical BNN inference with NUTS. The model follows the one from Radford's Neal Thesis

In [ ]:
import arviz as az
import numpy
import matplotlib.pyplot as plt
from cmdstanpy import CmdStanModel
import cmdstanpy
import torch
import os

# Uncomment and run
#cmdstanpy.install_cmdstan()

In [ ]:
## Choose model compilation (for this check the python file attached)
cpp_options = {}
stan_threads = None


In [ ]:
## Compile the program
sm = CmdStanModel(
                  stan_file   = './stan_files/Bayesian_Neural_Net_categorical_GLM_no_partial_sum.stan', 
                  cpp_options = cpp_options
                 )



In [ ]:
## Load the data
N = 100
C = 10

x = numpy.random.randn(N,C)
y = numpy.random.randint(C, size = (N,))

y += 1

In [ ]:
## Input data to the program

## neural net topology
num_hidden_layers     = 1
num_neurons_per_layer = 128

## prior definition
mu_w   = 0.0
    
BNN_data = {
                'N'           : N,
                'C'           : C,
                'x'           : x,
                'y'           : y,
                'mu_w'        : mu_w,
                'num_hidden'  : num_hidden_layers,
                'num_neurons' : num_neurons_per_layer
            }

output_dir = os.path.join('./results_hierarchical_bnn/')

In [ ]:
## ==========================
## Regarding Chain Generation

iter_sampling = 1000    # number of samples to draw after warm up
chains        = 3     # number of chains sampled in parallel
iter_warmup   = 300  # number of iterations of the warm up stage
thin          = 1     # chain thinning. Can improve ESS

## ====================
## NUTS sampler
max_treedepth = 10

## Step size adaptation
delta = 0.8  # acceptance rate probabilty 
# Below parameters are not available through the cmdstanpy interface 
# gamma = 0.05 # from nesterov algorithm. Stan recommends default
# kappa = 0.75 # from nesterov algorithm. Stan recommends default  
# t_0   = 10   # from nesterov algorithm. Stan recommends default


## Adaptation stage
adapt_engaged       = True  # If false no adaptation is done
adapt_init_phase    = 75 # correspond to adaptation step I   in Stan reference manual. Specifies width in samples
adapt_metric_window = 25 # correspond to adaptation step II  in Stan reference manual. Specifies width in samples
adapt_step_size     = 50 # correspond to adaptation step III in Stan reference manual. Specifies width in samples

if not adapt_engaged:
    delta               = None
    adapt_init_phase    = None
    adapt_metric_window = None
    adapt_step_size     = None
    iter_warmup         = None

## ====================
## Specification for kinetic energy
metric_M = 'diag_e' # the shape of the correlation matrix M. This is very important, check 2014 MLSS talk from Betancourt on youtube
                     # options diag_e. dense_e will handle correlations 


fit_sampler  = sm.sample(
                         # data passed to stan
                         data          = BNN_data, 
    
                         # ===
                         # specification about the chain generated
                         iter_sampling     = iter_sampling, 
                         chains            = chains, 
                         threads_per_chain = stan_threads, # number of threads per chain when STAN_THREADS are activated.
                                                # this threads are used to parallelize the reduce_sum computations        
    
                         iter_warmup   = iter_warmup, 
                         save_warmup   = True,  # warm up samples are kept or not
    
                         seed          = 1,
                    
                         # ===
                         # optimization of step size
                         adapt_delta = delta,
                         # gamma = gamma,
                         # kappa = kappa,
                         # t_0   = t_0,
                    
                         # ===
                         # adaptation stage
                         adapt_engaged       = adapt_engaged,           
                         adapt_init_phase    = adapt_init_phase,
                         adapt_metric_window = adapt_metric_window,
                         adapt_step_size     = adapt_step_size,

                         # ===
                         # Kinetic energy
                         metric = metric_M,
    
    
                         # ===
                         # Config things
                         output_dir    = output_dir,
                         show_progress = 'notebook',
                         save_diagnostics = True,
                         validate_csv = True
                        )


In [ ]:
#### =========================== ####
#### == Run CMDSTAN diagnoise == ####
print(fit_sampler.diagnose())


In [ ]:
#### =========================== ####
#### == Print Sampler summary == ####
print(fit_sampler.summary())

In [ ]:
#### ============================== ####
#### == Get Posterior Parameters == ####
# Posterior variables 
W_inp = fit_sampler.stan_variable(name = 'Winp')
W_out = fit_sampler.stan_variable(name = 'Wout')
W_h   = fit_sampler.stan_variable(name = 'Wh')

print(" === WEIGHTS === ")
print('W_inp', W_inp.shape)
print('W_out', W_out.shape)
print('W_h'  , W_h.shape)

print(" === Bias === ")
b_inp = fit_sampler.stan_variable(name = 'binp')
b_out = fit_sampler.stan_variable(name = 'bout')
b_h   = fit_sampler.stan_variable(name = 'bh')

print('b_inp', b_inp.shape)
print('b_out', b_out.shape)
print('b_h'  , b_h.shape)

print(" === Hyperparam ===")
sigma = fit_sampler.stan_variable(name = 'sigma')

print(sigma.shape)


In [ ]:
_ = plt.hist(sigma, bins = 50)